In [1]:
import numpy as np
import scipy as sp
import pandas as pd

Михайло Корешков  
магістр КАУ, 2 курс, спеціальність математика  

Статистика  
Домашнє завдання 2

## Завдання 1
Дано 16 вимірювань температури в цельсіях.
Знайти вибіркове середнє, вибіркову дисперсію, вибіркову медіану в цельсіях, кельвінах, фаренгейтах.

In [52]:
C = np.array([27, 26, 24, 22, 20, 19, 22, 22, 22, 27, 29, 29, 18, 14, 20, 22])
assert len(C) == 16
print("Все ок")

Все ок


Фаренгейти та Кельвіни обчислюються отак: 
```
F = C * 1.8 + 32
K = C + 273.15
```

Перевіряю реалізацію ключовими температурами: 
- абсолютний ноль
- точка замерзання солоної води нижча за звичайної води
- цельсій та фаренгейт співпадають на -40 

In [51]:
def c2f(c): return 1.8 * c + 32
def c2k(c): return c + 273.15

assert c2k(-273.15) == 0
assert c2f(0) > 0
assert c2f(-40) == -40
print("Все ок")


Все ок


In [5]:
F = c2f(C)
F

array([80.6, 78.8, 75.2, 71.6, 68. , 66.2, 71.6, 71.6, 71.6, 80.6, 84.2,
       84.2, 64.4, 57.2, 68. , 71.6])

In [6]:
K = c2k(C)
K

array([300.15, 299.15, 297.15, 295.15, 293.15, 292.15, 295.15, 295.15,
       295.15, 300.15, 302.15, 302.15, 291.15, 287.15, 293.15, 295.15])

Самостійна реалізація статистик, щоб перевірити, що я правильно розумію як вони обчислюються.
Перевіряю на випадковій вибірці парної і непарної довжини (щоб перевірити обчислення медіани).

In [50]:
def s_mean(X): 
    return X.sum() / len(X)
def s_var(X): 
    return ((X - s_mean(X))**2).sum() / len(X)
def s_med(X): 
    sorted = np.sort(X)
    a = int(np.floor((len(X)-1) / 2))
    b = int(np.ceil((len(X)-1) / 2))
    return (sorted[a] + sorted[b])/2

_rnd = np.random.randint(-100,100,16)
assert s_mean(_rnd) == _rnd.mean()
assert s_var(_rnd) == _rnd.var()
assert s_med(_rnd) == np.median(_rnd), f"{_rnd=}\n{s_med(_rnd)=} {np.median(_rnd)=}"

_rnd = np.random.randint(-100,100,17)
assert s_mean(_rnd) == _rnd.mean()
assert s_var(_rnd) == _rnd.var()
assert s_med(_rnd) == np.median(_rnd), f"{_rnd=}\n{s_med(_rnd)=} {np.median(_rnd)=}"
print("Все ок")

Все ок


надалі користуватимусь функціями numpy

In [44]:
print("Середнє, Цельсій: ", C.mean())
print("Середнє, Кельвін: ", K.mean())
print("Середнє, Фаренгейт: ", F.mean())

print("Дисперсія, Цельсій: ", C.var())
print("Дисперсія, Кельвін: ", K.var())
print("Дисперсія, Фаренгейт: ", F.var())

print("Медіана, Цельсій: ", np.median(C))
print("Медіана, Кельвін: ", np.median(K))
print("Медіана, Фаренгейт: ", np.median(F))

Середнє, Цельсій:  22.6875
Середнє, Кельвін:  295.8375
Середнє, Фаренгейт:  72.8375
Дисперсія, Цельсій:  16.08984375
Дисперсія, Кельвін:  16.08984375
Дисперсія, Фаренгейт:  52.13109375
Медіана, Цельсій:  22.0
Медіана, Кельвін:  295.15
Медіана, Фаренгейт:  71.6


Перевіримо деякі твердження про ці статистики.
1. Вибіркова медіана лінійна
2. Вибіркове середнє лінійне
3. Вибіркова дисперсія не змінюється при додаванні константи
4. Константи під вибірковою дисперсією при множенні виносяться у квадраті

In [49]:
assert np.median(K) == c2k(np.median(C))
assert np.median(F) == c2f(np.median(C))

assert K.mean() == c2k(C.mean())
assert F.mean() == c2f(C.mean())

assert K.var() == C.var()
assert np.abs(F.var() - C.var() * 1.8**2) < 1e-6

print("Все ок")

Все ок


## Завдання 2

$X_k - i.i.d$  
$P(X_1 = -1) = P(X_1 = 0) = \alpha$  
$P(X_1 = 1) = 1-2\alpha$  
$\alpha \in [0, 0.5]$  

Побудувати незміщену консистентну оцінку $\alpha$ за наступних умов

1. Відома вся вибірка 
2. Відоме лише вибіркове середнє
3. Відома лише кількість нулів
4. Відома лише величина $Y_n = \sum_{k=1}^nX^2_k$

Порівняти дисперсію


### a) Відома вся вибірка $X_1, ..., X_n$

(я робив це на практиці, але все одно почну спочатку)

Обчислю матсподівання $X$, може там щось корисне.

$MX = \sum_{x = -1,0,1} x P(X=x) = -\alpha + 0 \alpha + (1-2\alpha) = 1-3\alpha$  
$\alpha = (1-MX)/3$

$$MX = 1-3\alpha$$


Прекрасно, таким чином за методом моментів  
$\hat \alpha_1 = (1-m)/3$,  
де $m=\overline X$ 

$M\hat\alpha_1 = M\left((1-m)/3\right) = (1-M\overline X)/3 = (1 - (1-3\alpha))/3 = \alpha$, звісно. 

$$M\hat\alpha_1 = \alpha$$

Оцінка $\hat\alpha_1$ незміщена

Вибіркове середнє - це консистентна оцінка матсподівання, за законом великих чисел.
Консистентність $\hat\alpha_1$ отримуємо з того, що $f(x) = (1-x)/3$ є неперервною функцією, а неперервні функції зберігають збіжність за ймовірністю.

$\overline X_n \longrightarrow MX$ при $n\to\infty$  
$(1-\overline X_n)/3 \longrightarrow (1-MX)/3$ при $n\to\infty$  
$\hat\alpha_1 \longrightarrow \alpha$ при $n\to\infty$  

Отже, $\hat\alpha_1 = (1-\overline X)/3$ - незміщена та консистентна

$MX^2 = 1 \alpha + 0 \alpha + 1 (1-2\alpha) = 1-\alpha$

$DX = MX^2 - (MX)^2 = 1-\alpha - (1-6\alpha+9\alpha^2) = 5\alpha -9\alpha^2$

$$DX = 5\alpha - 9\alpha^2$$

$D\hat\alpha_1 = D\left((1-\overline X)/3\right) = \frac{1}{9} D(1-\overline X) = \frac{1}{9} D(\overline X) =
\frac{1}{9n} D(X)$  
$$D\hat\alpha_1 = \frac{5\alpha-9\alpha^2}{9n} = \frac{\frac{5}{9}\alpha-\alpha^2}{n}$$



### b) Відоме лише вибіркове середнє $\overline X_n$

Власне, оцінка $\hat\alpha_1 = (1-\overline X)/3$ нам підходить

### c) Відома лише кількість нулів у вибірці

$Z_n = \sum_{k=1}^n 1(X_k=0)$

Спробуємо обчислити матсподівання $Z_n$ і подивитись що буде.

$MZ_n = \sum_{k=1}^n P(X_k = 0) = n\alpha$  
$\alpha = MZ_n / n$

Можна помітити що частка нульових елементів точно співпадає з параметром $\alpha$, тобто нам підійде $\hat\alpha_3 = Z_n / n$.  
$M(\hat\alpha_3) = M(Z_n / n) = n\alpha / n = \alpha$

$$M\hat\alpha_3 = \alpha$$
Маємо незміщену оцінку

За законом великих чисел,  
$Z_n / n \longrightarrow P(X = 0) = \alpha$  
Отже, оцінка ще і консистентна.

$M1(x_k=0) = \alpha$    
$D1(x_k=0) = M(1(x_k=0) - \alpha)^2 = \alpha \cdot \alpha^2 + \alpha \cdot (1-\alpha)^2 + (1-2\alpha) \cdot \alpha^2 =$  
$= \alpha^3 + \alpha - 2\alpha^2 + \alpha^3 + \alpha^2 - 2\alpha^3 = \alpha - \alpha^2$

$D\hat\alpha_3 = D\bigr(\sum 1(x_k=0) / n \bigl) = \frac{D1(x_k=0)}{n} = \frac{\alpha-\alpha^2}{n}$

$$D\hat\alpha_3 = \frac{\alpha-\alpha^2}{n} \ge \frac{\frac{5}{9}\alpha-\alpha^2}{n} = D\hat\alpha_1$$


### d) Відома лише величина Y
$Y_n = \sum_{k=1}^n X^2_k$

Фактично, величина $Y$ дає кількість ненульових величин. 

$Y_n = \sum_{k=1}^n 1(X_k \ne 0)$

Але тоді  
$Y_n = n - Z_n$

І тоді можна отримати попередню оцінку,  
$\hat\alpha_3 = n - Z_n$